<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_9/Aula_9_1k_Queries_Positive_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = 'Leonardo Augusto da Silva Pacheco'
print(f'Meu nome é {nome}')

Meu nome é Leonardo Augusto da Silva Pacheco


# Enunciado

https://github.com/leonardo3108/IA368dd/blob/main/exercicios/Aula_9/enunciado.md

# Setup

## Instalação de libs

In [28]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação de libs

In [34]:
import json
import os
import openai
import numpy as np

## Chave de acesso à API OpenAI

In [32]:
openai.api_key = 'API_KEY_OPENAI'

# Preparação dos dados

## Obtenção - TREC-COVID

In [4]:
!wget -nc 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

--2023-05-02 00:55:11--  https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip
Resolving public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)... 130.83.167.186
Connecting to public.ukp.informatik.tu-darmstadt.de (public.ukp.informatik.tu-darmstadt.de)|130.83.167.186|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73876720 (70M) [application/zip]
Saving to: ‘trec-covid.zip’

trec-covid.zip      100%[===================>]  70.45M  11.9MB/s    in 7.4s    

2023-05-02 00:55:20 (9.47 MB/s) - ‘trec-covid.zip’ saved [73876720/73876720]



In [5]:
!unzip -o trec-covid.zip

Archive:  trec-covid.zip
   creating: trec-covid/
   creating: trec-covid/qrels/
  inflating: trec-covid/qrels/test.tsv  
  inflating: trec-covid/corpus.jsonl  
  inflating: trec-covid/queries.jsonl  


## Tratamento - qrels de test

In [6]:
with open('trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)

print(data[:5])
print(data[4].split())

['query-id\tcorpus-id\tscore\n', '1\t005b2j4b\t2\n', '1\t00fmeepz\t1\n', '1\tg7dhmyyo\t2\n', '1\t0194oljo\t1\n']
['1', '0194oljo', '1']


In [7]:
with open('test_adjusted.tsv', 'w') as fout:
    for line in data[1:]:
        query_id, corpus_id, score = line.split()
        fout.write(f'{query_id}\t0\t{corpus_id}\t{score}\n')

## Tratamento - corpus

In [9]:
corpus = []
with open('trec-covid/corpus.jsonl') as fin:
    for i, line in enumerate(fin):
        corpus.append(json.loads(line))

for text in corpus[:10]:
    print(text)

{'_id': 'ug7v899j', 'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60

## Tratamento - queries

In [ ]:
queries = []
with open('trec-covid/queries.jsonl') as fin:
    for line in fin:
      query = json.loads(line)
      queries.append({'id': query['_id'], 'text': query['text']})

for query in queries[:10]:
    print(query)      

{'id': '1', 'text': 'what is the origin of COVID-19'}
{'id': '2', 'text': 'how does the coronavirus respond to changes in the weather'}
{'id': '3', 'text': 'will SARS-CoV2 infected people develop immunity? Is cross protection possible?'}
{'id': '4', 'text': 'what causes death from Covid-19?'}
{'id': '5', 'text': 'what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?'}
{'id': '6', 'text': 'what types of rapid testing for Covid-19 have been developed?'}
{'id': '7', 'text': 'are there serological tests that detect antibodies to coronavirus?'}
{'id': '8', 'text': 'how has lack of testing availability led to underreporting of true incidence of Covid-19?'}
{'id': '9', 'text': 'how has COVID-19 affected Canada'}
{'id': '10', 'text': 'has social distancing had an impact on slowing the spread of COVID-19?'}


# Geração de 1000 queries relevantes

## Geração randômica de índices

In [87]:
np.random.seed(10)

indexes_base = np.random.choice(len(corpus), 10000, replace=False)
indexes = []
i = 0
for idx in indexes_base:
    text = corpus[idx]['text']
    if len(text) > 200 and len(text) < 1000 and 'the' in text:
        indexes.append(idx)
        i += 1
    if i == 1000:
        break
assert len(indexes) == 1000
print(indexes)

[26293, 92263, 36811, 22178, 79286, 50058, 115985, 117163, 63121, 80260, 159866, 152100, 24940, 82346, 41170, 160849, 155573, 9853, 42500, 131684, 113694, 562, 93238, 148433, 132054, 121966, 157344, 36129, 156024, 132029, 112774, 22815, 8613, 164002, 139235, 161012, 12638, 76607, 119945, 50698, 21625, 154788, 17667, 14142, 64843, 159856, 101559, 77379, 128269, 169549, 94123, 166344, 31933, 1842, 124612, 78777, 119854, 87019, 47145, 46222, 119991, 126679, 34798, 118758, 133015, 100507, 134882, 168524, 39835, 66386, 71493, 32021, 119208, 149017, 160935, 96437, 6080, 168298, 170034, 135914, 157666, 165078, 160631, 22626, 109094, 162003, 40596, 124289, 21182, 160842, 80018, 144570, 116227, 153917, 78133, 24441, 49083, 84727, 115427, 141785, 5168, 168593, 62441, 116487, 131250, 80831, 146949, 88564, 8697, 26541, 72833, 107770, 165683, 72592, 61116, 150454, 45784, 68569, 127249, 129714, 113840, 147452, 168362, 22407, 21666, 124394, 128722, 167370, 30169, 13836, 30894, 129725, 154616, 148024,

## Geração das queries

In [131]:
with open('leonardo_pacheco_1k_generated_queries_20230501.jsonl', 'w', encoding='utf-8') as fout:
    for i, idx in enumerate(indexes):
        text = corpus[idx]['text']
        prompt = f'''Based on the following text, give me exactly one query that I could use in an internet search:
Text: Theoretically, there are several ways of preventing the common cold: quarantine, immunisation (or vaccination); early treatment of effected individuals; or physical barriers to reduce transmission.
Query: How to prevent common cold transmission?

Text: What are the results of current COVID-19 treatment trials?
Query: The following systematic review and meta-analysis compile the current data regarding human controlled COVID-19 treatment trials. An electronic search of the literature compiled studies pertaining to human controlled treatment trials with COVID-19.

Text: What FDA-approved drugs inhibit coronavirus replication in cell culture?
Query: We have screened a library of 348 FDA-approved drugs for anti-coronaviroses activity in cell culture.

Text: {text}
Query: '''
        #print(prompt)
        response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                    messages=[
                        {'role': 'system', 'content': 'You''re cataloging documents and need to associate queries with document passages.'},
                        {'role': 'user', 'content': prompt}
                    ],
                    temperature=0,
                    max_tokens=500)
        new_doc = {'positive_doc_id': corpus[idx]['_id'], 'query': response['choices'][0]['message']['content'].strip('"').rstrip('.')}
        fout.write(f'{json.dumps(new_doc)}\n')
        if (i + 1) % 100 == 0:
            print(f'{i+1} documentos processados')

100 documentos processados
200 documentos processados
300 documentos processados
400 documentos processados
500 documentos processados
600 documentos processados
700 documentos processados
800 documentos processados
900 documentos processados
1000 documentos processados


In [134]:
!head leonardo_pacheco_1k_generated_queries_20230501.jsonl

{"positive_doc_id": "cb63iyr2", "query": "Optimization algorithm for multi-aisle automatic storage/retrieval system scheduling"}
{"positive_doc_id": "bh717utp", "query": "What is the basic reproduction number of the novel coronavirus outbreak in Wuhan, China?"}
{"positive_doc_id": "4i8pepto", "query": "ABCD vaccination guidelines for cats"}
{"positive_doc_id": "y5h942ar", "query": "How does the CLEARSY Safety Platform work for developing safety critical applications?"}
{"positive_doc_id": "6v5niqxv", "query": "First documented case of morbillivirus infection in a wild Siberian tiger"}
{"positive_doc_id": "4hu92j8w", "query": "How does the COVID-19 crisis affect the mental health of older adults and what measures can be taken to prevent suicide?"}
{"positive_doc_id": "1d7umc0k", "query": "Interaction between S2 subunit protein of SARS-nCoV-2 and tumor suppressor proteins p53 and BRCA-1/2"}
{"positive_doc_id": "x8cjyscw", "query": "Cardiovascular complications in COVID-19 patients"}
{"po

In [135]:
!wc -l leonardo_pacheco_1k_generated_queries_20230501.jsonl

1000 leonardo_pacheco_1k_generated_queries_20230501.jsonl
